In addition to the original script, 2_generate_TP, this script does a blast search of selected_tp_genes against temp_neg_genomes and filters out all genomes that have a hit. In addition to 2.1_generate_TP, this script also does an hmm search, using an alignment of selected_tp_genes as query against negative genomes. It combines results of blast and hmm searches and removes all samples with hits from the pool of neg_genomes to choose from. One further parameter is required to specify an hmm evalue cutoff for samples to remove. Ideally, this would be 0, but I decided to keep a cutoff, as the original negative genomes from the sugimoto paper also have positive hits for an oxyN hmm.


This script generates the 2 remaining files required for input:
- True positive genes (nucleotide fasta)
- protein alignment (amino acid fasta)
IMPORTANTLY, these 2 files are created from non-overlapping sequences!


And also moves the corresponding files to new directories so that coverage tables can be generated from them.

In the first cell specify:
- BGC type (This name must stay constant throughout the scripts)
- select_neg_genomes, i.e. the amount of negative genomes to be transferred to the neg_genomes directory
- select_pos_genomes, i.e. the amount of positive genomes to be transferred to the pos_genomes directory and to generate the tp_genes file from (the surplus amount will be used to generate the protein alignment from)
- pos_isolation_source_filter, if these terms are found in the isolation_source column of the positive samples in the summary file, they will be scored higher in a scoring column, i.e. samples from a known and desired isolation source will be used preferentially.
- neg_isolation_source_filter, accordingly
- avoid_list. These terms are scored with a 0, end at the bottom of the table, and will be picked last. This is useful when an uncommon gene is searched for and more, and/or more tenuous isolation sources have been allowed during download. These are generally words that contain one of the search terms, e.g. 'sea' in 'diseased'.

Modify in such a way that TP genes are used as query against all individual negative genomes. Negative genomes are only moved from temp directory to neg_genomes directory if the blast search comes back negative

In [1]:
BGC_type = 'RTX_toxin_acyltransferase'
select_neg_genomes = 140
select_pos_genomes = 10
hmm_evalue_cutoff = 3

avoid_list = ['', 'isolation_source not annotated', 'diseased', 'mice', 'spice', 'septicemic', 'research', 'crevice']
#these are identical to first script, but don't have to be
pos_isolation_source_filter =  ['marine', 'sea', 'sponge', 'ocean', 'porifera', 'seafloor','sediment', 'water', 'tidal', 'coral', 'reef', 'coast', 'ship', 'fish', 'aquaculture', 'atlantic', 'pacific', 'mediterranean', 'baltic', 'pond', 'river', 'ice', 'carribean', 'lake', 'fjord', 'marina', 'hydro', 'algal', 'algae', 'clam', 'shell', 'mussel']
neg_isolation_source_filter = ['marine', 'sea', 'sponge', 'ocean', 'porifera', 'seafloor', 'sediment', 'water', 'tidal', 'coral', 'reef', 'coast', 'ship', 'fish', 'aquaculture', 'atlantic', 'pacific', 'mediterranean', 'baltic', 'pond', 'river', 'ice', 'carribean', 'lake', 'fjord', 'marina', 'hydro', 'algal', 'algae', 'clam', 'shell', 'mussel']

In [2]:
import os
from os import listdir, mkdir
from os.path import isfile, join
from pathlib import Path
import pandas as pd
from pandas.errors import EmptyDataError
import warnings

In [3]:
def makedir(dirpath):
    if os.path.isdir(dirpath):
        print(dirpath,'exists already')
    else:
        print('Making', dirpath)    
        os.mkdir(dirpath)

        
# Defining paths for required directory structure for input and output files relative to parent directory
#parent_dir='/media/manu/RiPP_Prioritiser/'
#will make directories relative to the path the notebook was opened in
parent_dir= !echo $(pwd)
BGC_path=os.path.join(parent_dir[0], BGC_type)
neg_genomes_path=os.path.join(BGC_path, 'base_genomes/temp_neg_genomes')
pos_genomes_path=os.path.join(BGC_path, 'base_genomes/temp_pos_genomes')
output_neg_path=os.path.join(BGC_path, 'base_genomes/neg_genomes')
output_pos_path=os.path.join(BGC_path, 'base_genomes/pos_genomes')
#qc-paths.
quality_control_path=os.path.join(BGC_path, 'base_genomes/quality_control')
neg_blast_path=os.path.join(quality_control_path, 'neg_blast')
neg_blast_db_path=os.path.join(neg_blast_path, 'databases')
neg_blast_results_path=os.path.join(neg_blast_path, 'outfiles')
pos_blast_path=os.path.join(quality_control_path, 'pos_blast')
pos_blast_db_path=os.path.join(pos_blast_path, 'databases')
pos_blast_results_path=os.path.join(pos_blast_path, 'results')
neg_hmm_path=os.path.join(quality_control_path, 'neg_hmm')
neg_hmm_db_path=os.path.join(neg_hmm_path, 'queries')
neg_hmm_results_path=os.path.join(neg_hmm_path, 'outfiles')
pos_hmm_path=os.path.join(quality_control_path, 'pos_hmm')
pos_hmm_db_path=os.path.join(pos_hmm_path, 'databases')
pos_hmm_results_path=os.path.join(pos_hmm_path, 'results')

# made directories manually. remove or comment after running.
#validation_pos_genomes_path=os.path.join(BGC_path, 'base_genomes/validation_pos_genomes')
#validation_coverage_table_path=os.path.join(BGC_path, 'validation_coverage_tables')

# Calling function to make directories if they don't exist yet
makedir(output_neg_path)
makedir(output_pos_path)
makedir(quality_control_path)
makedir(neg_blast_path)
makedir(neg_blast_db_path)
makedir(neg_blast_results_path)
makedir(pos_blast_path)
makedir(pos_blast_db_path)
makedir(pos_blast_results_path)
makedir(neg_hmm_path)
makedir(neg_hmm_db_path)
makedir(neg_hmm_results_path)
makedir(pos_hmm_path)
makedir(pos_hmm_db_path)
makedir(pos_hmm_results_path)

#makedir(validation_pos_genomes_path)

os.chdir(BGC_path)

Making /media/manu/RiPP_Prioritiser/RTX_toxin_acyltransferase/base_genomes/neg_genomes
Making /media/manu/RiPP_Prioritiser/RTX_toxin_acyltransferase/base_genomes/pos_genomes
Making /media/manu/RiPP_Prioritiser/RTX_toxin_acyltransferase/base_genomes/quality_control
Making /media/manu/RiPP_Prioritiser/RTX_toxin_acyltransferase/base_genomes/quality_control/neg_blast
Making /media/manu/RiPP_Prioritiser/RTX_toxin_acyltransferase/base_genomes/quality_control/neg_blast/databases
Making /media/manu/RiPP_Prioritiser/RTX_toxin_acyltransferase/base_genomes/quality_control/neg_blast/outfiles
Making /media/manu/RiPP_Prioritiser/RTX_toxin_acyltransferase/base_genomes/quality_control/pos_blast
Making /media/manu/RiPP_Prioritiser/RTX_toxin_acyltransferase/base_genomes/quality_control/pos_blast/databases
Making /media/manu/RiPP_Prioritiser/RTX_toxin_acyltransferase/base_genomes/quality_control/pos_blast/results
Making /media/manu/RiPP_Prioritiser/RTX_toxin_acyltransferase/base_genomes/quality_control/n

In [4]:
# Generating a report file for this script
with open(BGC_path+'/'+'report_2_generate_tp.txt', 'w') as f:
    f.write('Output directory is: '+BGC_path+'\n')
    f.write('\nBGC_type = '+BGC_type)
    f.write('\nselect_neg_genomes = '+str(select_neg_genomes))
    f.write('\nselect_pos_genomes = '+str(select_pos_genomes))
    f.write('\nhmm_evalue_cutoff = '+str(hmm_evalue_cutoff))
    f.write('\navoid_list = '+str(avoid_list))
    f.write('\nneg_isolation_source_filter = '+str(neg_isolation_source_filter))
    f.write('\npos_isolation_source_filter = '+str(pos_isolation_source_filter)+'\n')

In [5]:
# load summary table into data frame () output from 1.)
summary_file = pd.read_csv('summary.tsv', sep='\t')

Change order of tables to prioritize samples that have an isolation source

In [6]:
warnings.filterwarnings('ignore')

#filter positives and drop all duplicate protein sequences originating from different organisms
pos_mask = (summary_file['dir'] == '+')
pos_df = summary_file[pos_mask]
pos_df.drop_duplicates(subset='protein_id', keep=False, inplace=True)


#filter negatives
neg_mask = (summary_file['dir'] == '-')
neg_df = summary_file[neg_mask]

#scoring words in isolation source so as to preferentially pick samples with chosen isolation sources

def custom_sorting(source,isolation_source_filter):
    score = 1
    if isolation_source_filter=='pos':
        for word in pos_isolation_source_filter:
            if word in source:
                score +=1
        for word in avoid_list:
            if source == word:
                score=0
    elif isolation_source_filter=='neg':
        for word in neg_isolation_source_filter:
            if word in source:
                score +=1
        for word in avoid_list:
            if source == word:
                score=0
    return score


pos_df['scoring_column'] = pos_df.apply(lambda x: custom_sorting(x['isolation_source'],'pos'),axis=1)
neg_df['scoring_column'] = neg_df.apply(lambda x: custom_sorting(x['isolation_source'],'neg'),axis=1)

pos_df.sort_values(by=['scoring_column'], axis=0, ascending=False, inplace=True)
neg_df.sort_values(by=['scoring_column'], axis=0, ascending=False, inplace=True)

In [7]:
#Split positive genomes into 2 bins, one goes towards tp-genes and is the pos-genomes used for synthesising metagenomes
#the other one constitutes a source of protein sequences for alignment as an input file
#(removed the latter functionality and used pfam alignments of the protein instead)

# Genomes selected in such a way that they are from the top of the pre-sorted pos_df
unique_pos_df = pos_df.drop_duplicates(subset='assembly', inplace=False)
selected_tp_genomes = list(unique_pos_df.iloc[:,1])[0:select_pos_genomes]
remaining_pos_genomes = list(unique_pos_df.iloc[:,1])[select_pos_genomes:]

with open(BGC_path+'/'+'report_2_generate_tp.txt', 'a') as f:
    f.write('\nselected_tp_genomes are:\n')
    f.write(str(selected_tp_genomes)+'\n')

#select genomes and isolate GCF number from them, move selected tp genomes to final pos_genomes directory
for genome in selected_tp_genomes:
    print('moving positive', genome, 'to', output_pos_path)
    !mv "{pos_genomes_path}"/"{genome}"* "{output_pos_path}"
    
#generate dataframe containing all tp-genomes and all the tp-genes contained in it
filtered_pos_df = pos_df[pos_df['assembly'].isin(selected_tp_genomes)]
remaining_pos_df = pos_df[~pos_df['assembly'].isin(selected_tp_genomes)]

#isolate all the headers and transfer them to the selected_tp_genes file
full_header_list = []
for i in range(0,len(filtered_pos_df)):
    full_header=str('>')+filtered_pos_df.iloc[i,1]+str('_')+filtered_pos_df.iloc[i,3]+str('_')+filtered_pos_df.iloc[i,5]
    full_header_list.append(full_header)

# generate fasta file with selected tp genes found in the selected genomes
print('generating selected_tp_genes.fasta')
with open(BGC_path+'/'+'report_2_generate_tp.txt', 'a') as f:
    f.write('\nselected_tp_genes in positive genomes are:\n')
tp_gene_counter=0
with open(BGC_path+'/'+BGC_type+'_tp_genes.fasta') as fh:
    lines=fh.readlines()
    for i in range(0,len(lines)):
        for j in range(0,len(full_header_list)):
            if full_header_list[j] in lines[i]:
                tp_gene_counter+=1
                with open(BGC_path+'/'+'report_2_generate_tp.txt', 'a') as f:
                    f.write(lines[i][1:-1]+'\n')
                with open(BGC_path+'/'+BGC_type+'_selected_tp_genes.fasta', 'a') as outfile:
                    outfile.write(lines[i]+lines[i+1])    
with open(BGC_path+'/'+'report_2_generate_tp.txt', 'a') as f:
    f.write('\n'+str(len(selected_tp_genomes))+' unique genomes with '+ str(tp_gene_counter)+' unique tp genes.\n\n')
                    
                    
# transfer all amino acid sequences that are not part of the tp-genomes to a fasta file
print('generating selected_tp_aa.fasta')
with open(BGC_path+'/'+'report_2_generate_tp.txt', 'a') as f:
    f.write('\nsremaining samples are:\n')
tp_aa_counter = 0
with open(BGC_path+'/'+BGC_type+'_selected_tp_aa.fasta', 'a') as outfile:
    for i in range(0,len(remaining_pos_df)):
        tp_aa_counter+=1
        fasta_header=str('>')+remaining_pos_df.iloc[i,1]+str('_')+remaining_pos_df.iloc[i,3]+str('_')+remaining_pos_df.iloc[i,5]+'\n'
        sequence = remaining_pos_df.iloc[i,6][2:-2]+'\n'
        with open(BGC_path+'/'+'report_2_generate_tp.txt', 'a') as f:
            f.write(fasta_header[1:-1]+'\n')
        outfile.write(fasta_header)
        outfile.write(sequence)
with open(BGC_path+'/'+'report_2_generate_tp.txt', 'a') as f:
    f.write('\n'+str(len(remaining_pos_genomes))+' unique genomes with '+ str(tp_aa_counter)+' unique aa sequences.\n\n')

    
print('Done')

moving positive GCF_013357885.1 to /media/manu/RiPP_Prioritiser/RTX_toxin_acyltransferase/base_genomes/pos_genomes
moving positive GCF_002119525.1 to /media/manu/RiPP_Prioritiser/RTX_toxin_acyltransferase/base_genomes/pos_genomes
moving positive GCF_000307145.1 to /media/manu/RiPP_Prioritiser/RTX_toxin_acyltransferase/base_genomes/pos_genomes
moving positive GCF_013357825.1 to /media/manu/RiPP_Prioritiser/RTX_toxin_acyltransferase/base_genomes/pos_genomes
moving positive GCF_008369605.1 to /media/manu/RiPP_Prioritiser/RTX_toxin_acyltransferase/base_genomes/pos_genomes
generating selected_tp_genes.fasta
generating selected_tp_aa.fasta
Done


In [8]:
# move 10 more pos_genomes to validation_pos_genomes
#unique_pos_df = pos_df.drop_duplicates(subset='assembly', inplace=False)
#selected_tp_genomes = list(unique_pos_df.iloc[:,1])[0:select_pos_genomes]
#validation_tp_genomes=list(unique_pos_df.iloc[:,1])[select_pos_genomes:select_pos_genomes+10]

#select genomes and isolate GCF number from them, move selected tp genomes to final pos_genomes directory
#for genome in validation_tp_genomes:
#    print('moving positive', genome, 'to', validation_pos_genomes_path)
#    !mv "{pos_genomes_path}"/"{genome}"* "{validation_pos_genomes_path}"

In [9]:
# To confirm, run blast on all samples selected as validation pos genomes

# makes blast databases of all individual neg genomes (easier to keep track of accession numbers than when combining)
#for genomes in validation_tp_genomes:
#    !makeblastdb -in "{validation_tp_genomes}"/"{genomes}"* -dbtype nucl -out "{pos_blast_db_path}"/"{genomes}"_db

#for genomes in selected_tp_genomes:
#    !blastn -db "{pos_blast_db_path}"/"{genomes}"_db -query "{BGC_path}"/"{BGC_type}"_selected_tp_genes.fasta -out "{pos_blast_results_path}"/"{genomes}".blastout -outfmt "6 qseqid sseqid pident evalue length"
    
# use pandas to concatenate all blast output tables
#df_list = []
#for outfile in os.listdir(pos_blast_results_path):
#    try:
#        blast_df = pd.read_csv(pos_blast_results_path+'/'+outfile, sep='\t', names=['qseqid', 'sseqid', 'pident', 'evalue', 'length'], index_col=None)
#        blast_df['sample'] = '.'.join(outfile.split('.')[0:2])
#        df_list.append(blast_df)
#    except EmptyDataError:
#        continue
        
# Generate a list of contaminated negative genomes        
#pos_blast_hits_df = pd.concat(df_list)
#pos_blast_hits_df.to_csv(pos_blast_path+'/blast_results_summary.csv', index=False)

In [10]:
#Use blast to find tp_gene contamination in negative samples
unique_neg_df = neg_df.drop_duplicates(subset='assembly', inplace=False)
# gets a list of length of specified amount of neg genomes
neg_genomes_list = list(unique_neg_df.iloc[:,1])

with open(BGC_path+'/'+'report_2_generate_tp.txt', 'a') as f:
    f.write('\n'+'Using blastn to check negative genomes for contamination'+'\n')

# makes blast databases of all individual neg genomes (easier to keep track of accession numbers than when combining)
for genomes in neg_genomes_list:
    !makeblastdb -in "{neg_genomes_path}"/"{genomes}"* -dbtype nucl -out "{neg_blast_db_path}"/"{genomes}"_db

# runs blastn search of all TP genes against all blast databases of negative genomes
# even if no alteration is made to the original tp_genes file, rengame it to fit the path in the command below, accordingly.
for genomes in neg_genomes_list:
    !blastn -db "{neg_blast_db_path}"/"{genomes}"_db -query "{BGC_path}"/"{BGC_type}"_selected_tp_genes_altered.fasta -out "{neg_blast_results_path}"/"{genomes}".blastout -outfmt "6 qseqid sseqid pident evalue length"

# use pandas to concatenate all blast output tables
df_list = []
for outfile in os.listdir(neg_blast_results_path):
    try:
        blast_df = pd.read_csv(neg_blast_results_path+'/'+outfile, sep='\t', names=['qseqid', 'sseqid', 'pident', 'evalue', 'length'], index_col=None)
        blast_df['sample'] = '.'.join(outfile.split('.')[0:2])
        df_list.append(blast_df)
    except EmptyDataError:
        continue
        
# Generate a list of contaminated negative genomes        
remove_df = pd.concat(df_list)
remove_df.to_csv(quality_control_path+'/blast_fn_samples.csv', index=False)
unique_remove_df = remove_df.drop_duplicates(subset='sample', inplace=False)
blast_remove_list = list(unique_remove_df.iloc[:,4])

with open(BGC_path+'/'+'report_2_generate_tp.txt', 'a') as f:
    f.write('\n'+'Contamination with tp_seqs:\n'+str(list(remove_df.iloc[:,0]))+'\nsequences identified in negative samples:\n'+str(blast_remove_list)+'\n\n')

print('Done')



Building a new DB, current time: 11/09/2021 11:19:36
New DB name:   /media/manu/RiPP_Prioritiser/RTX_toxin_acyltransferase/base_genomes/quality_control/neg_blast/databases/GCF_003075095.1_db
New DB title:  /media/manu/RiPP_Prioritiser/RTX_toxin_acyltransferase/base_genomes/temp_neg_genomes/GCF_003075095.1_ASM307509v1_genomic.fna
Sequence type: Nucleotide
Keep MBits: T
Maximum file size: 1000000000B
Adding sequences from FASTA; added 87 sequences in 0.327964 seconds.


Building a new DB, current time: 11/09/2021 11:19:36
New DB name:   /media/manu/RiPP_Prioritiser/RTX_toxin_acyltransferase/base_genomes/quality_control/neg_blast/databases/GCF_013282925.1_db
New DB title:  /media/manu/RiPP_Prioritiser/RTX_toxin_acyltransferase/base_genomes/temp_neg_genomes/GCF_013282925.1_ASM1328292v1_genomic.fna
Sequence type: Nucleotide
Keep MBits: T
Maximum file size: 1000000000B
Adding sequences from FASTA; added 21 sequences in 0.161026 seconds.


Building a new DB, current time: 11/09/2021 11:19:3

Adding sequences from FASTA; added 922 sequences in 0.177863 seconds.


Building a new DB, current time: 11/09/2021 11:19:42
New DB name:   /media/manu/RiPP_Prioritiser/RTX_toxin_acyltransferase/base_genomes/quality_control/neg_blast/databases/GCF_003797885.1_db
New DB title:  /media/manu/RiPP_Prioritiser/RTX_toxin_acyltransferase/base_genomes/temp_neg_genomes/GCF_003797885.1_ASM379788v1_genomic.fna
Sequence type: Nucleotide
Keep MBits: T
Maximum file size: 1000000000B
Adding sequences from FASTA; added 45 sequences in 0.138212 seconds.


Building a new DB, current time: 11/09/2021 11:19:42
New DB name:   /media/manu/RiPP_Prioritiser/RTX_toxin_acyltransferase/base_genomes/quality_control/neg_blast/databases/GCF_003975335.1_db
New DB title:  /media/manu/RiPP_Prioritiser/RTX_toxin_acyltransferase/base_genomes/temp_neg_genomes/GCF_003975335.1_ASM397533v1_genomic.fna
Sequence type: Nucleotide
Keep MBits: T
Maximum file size: 1000000000B
Adding sequences from FASTA; added 42 sequences in 0.

Adding sequences from FASTA; added 22 sequences in 0.067265 seconds.


Building a new DB, current time: 11/09/2021 11:19:48
New DB name:   /media/manu/RiPP_Prioritiser/RTX_toxin_acyltransferase/base_genomes/quality_control/neg_blast/databases/GCF_900537165.1_db
New DB title:  /media/manu/RiPP_Prioritiser/RTX_toxin_acyltransferase/base_genomes/temp_neg_genomes/GCF_900537165.1_ASM90053716v1_genomic.fna
Sequence type: Nucleotide
Keep MBits: T
Maximum file size: 1000000000B
Adding sequences from FASTA; added 34 sequences in 0.171132 seconds.


Building a new DB, current time: 11/09/2021 11:19:49
New DB name:   /media/manu/RiPP_Prioritiser/RTX_toxin_acyltransferase/base_genomes/quality_control/neg_blast/databases/GCF_002786285.1_db
New DB title:  /media/manu/RiPP_Prioritiser/RTX_toxin_acyltransferase/base_genomes/temp_neg_genomes/GCF_002786285.1_ASM278628v1_genomic.fna
Sequence type: Nucleotide
Keep MBits: T
Maximum file size: 1000000000B
Adding sequences from FASTA; added 23 sequences in 0

Adding sequences from FASTA; added 4 sequences in 0.129736 seconds.


Building a new DB, current time: 11/09/2021 11:19:54
New DB name:   /media/manu/RiPP_Prioritiser/RTX_toxin_acyltransferase/base_genomes/quality_control/neg_blast/databases/GCF_002737765.1_db
New DB title:  /media/manu/RiPP_Prioritiser/RTX_toxin_acyltransferase/base_genomes/temp_neg_genomes/GCF_002737765.1_ASM273776v1_genomic.fna
Sequence type: Nucleotide
Keep MBits: T
Maximum file size: 1000000000B
Adding sequences from FASTA; added 406 sequences in 0.179132 seconds.


Building a new DB, current time: 11/09/2021 11:19:55
New DB name:   /media/manu/RiPP_Prioritiser/RTX_toxin_acyltransferase/base_genomes/quality_control/neg_blast/databases/GCF_004196115.1_db
New DB title:  /media/manu/RiPP_Prioritiser/RTX_toxin_acyltransferase/base_genomes/temp_neg_genomes/GCF_004196115.1_ASM419611v1_genomic.fna
Sequence type: Nucleotide
Keep MBits: T
Maximum file size: 1000000000B
Adding sequences from FASTA; added 147 sequences in 0.

Adding sequences from FASTA; added 149 sequences in 0.164984 seconds.


Building a new DB, current time: 11/09/2021 11:20:00
New DB name:   /media/manu/RiPP_Prioritiser/RTX_toxin_acyltransferase/base_genomes/quality_control/neg_blast/databases/GCF_000497995.1_db
New DB title:  /media/manu/RiPP_Prioritiser/RTX_toxin_acyltransferase/base_genomes/temp_neg_genomes/GCF_000497995.1_TB25_genomic.fna
Sequence type: Nucleotide
Keep MBits: T
Maximum file size: 1000000000B
Adding sequences from FASTA; added 458 sequences in 0.113337 seconds.


Building a new DB, current time: 11/09/2021 11:20:00
New DB name:   /media/manu/RiPP_Prioritiser/RTX_toxin_acyltransferase/base_genomes/quality_control/neg_blast/databases/GCF_002876165.1_db
New DB title:  /media/manu/RiPP_Prioritiser/RTX_toxin_acyltransferase/base_genomes/temp_neg_genomes/GCF_002876165.1_ASM287616v1_genomic.fna
Sequence type: Nucleotide
Keep MBits: T
Maximum file size: 1000000000B
Adding sequences from FASTA; added 169 sequences in 0.09989

...

Adding sequences from FASTA; added 6 sequences in 2.1083 seconds.


Building a new DB, current time: 11/09/2021 11:29:38
New DB name:   /media/manu/RiPP_Prioritiser/RTX_toxin_acyltransferase/base_genomes/quality_control/neg_blast/databases/GCF_019336185.1_db
New DB title:  /media/manu/RiPP_Prioritiser/RTX_toxin_acyltransferase/base_genomes/temp_neg_genomes/GCF_019336185.1_ASM1933618v1_genomic.fna
Sequence type: Nucleotide
Keep MBits: T
Maximum file size: 1000000000B
Adding sequences from FASTA; added 64 sequences in 2.63267 seconds.


Building a new DB, current time: 11/09/2021 11:29:41
New DB name:   /media/manu/RiPP_Prioritiser/RTX_toxin_acyltransferase/base_genomes/quality_control/neg_blast/databases/GCF_006372545.1_db
New DB title:  /media/manu/RiPP_Prioritiser/RTX_toxin_acyltransferase/base_genomes/temp_neg_genomes/GCF_006372545.1_ASM637254v1_genomic.fna
Sequence type: Nucleotide
Keep MBits: T
Maximum file size: 1000000000B
Adding sequences from FASTA; added 203 sequences in 2.152

Adding sequences from FASTA; added 100 sequences in 0.185658 seconds.


Building a new DB, current time: 11/09/2021 11:29:58
New DB name:   /media/manu/RiPP_Prioritiser/RTX_toxin_acyltransferase/base_genomes/quality_control/neg_blast/databases/GCF_006379415.2_db
New DB title:  /media/manu/RiPP_Prioritiser/RTX_toxin_acyltransferase/base_genomes/temp_neg_genomes/GCF_006379415.2_ASM637941v2_genomic.fna
Sequence type: Nucleotide
Keep MBits: T
Maximum file size: 1000000000B
Adding sequences from FASTA; added 48 sequences in 0.115958 seconds.


Building a new DB, current time: 11/09/2021 11:29:58
New DB name:   /media/manu/RiPP_Prioritiser/RTX_toxin_acyltransferase/base_genomes/quality_control/neg_blast/databases/GCF_013618635.1_db
New DB title:  /media/manu/RiPP_Prioritiser/RTX_toxin_acyltransferase/base_genomes/temp_neg_genomes/GCF_013618635.1_ASM1361863v1_genomic.fna
Sequence type: Nucleotide
Keep MBits: T
Maximum file size: 1000000000B
Adding sequences from FASTA; added 48 sequences in 0

Adding sequences from FASTA; added 276 sequences in 0.0859201 seconds.


Building a new DB, current time: 11/09/2021 11:30:08
New DB name:   /media/manu/RiPP_Prioritiser/RTX_toxin_acyltransferase/base_genomes/quality_control/neg_blast/databases/GCF_001721285.1_db
New DB title:  /media/manu/RiPP_Prioritiser/RTX_toxin_acyltransferase/base_genomes/temp_neg_genomes/GCF_001721285.1_PanISTKB1.0_genomic.fna
Sequence type: Nucleotide
Keep MBits: T
Maximum file size: 1000000000B
Adding sequences from FASTA; added 115 sequences in 0.188472 seconds.


Building a new DB, current time: 11/09/2021 11:30:09
New DB name:   /media/manu/RiPP_Prioritiser/RTX_toxin_acyltransferase/base_genomes/quality_control/neg_blast/databases/GCF_002330425.1_db
New DB title:  /media/manu/RiPP_Prioritiser/RTX_toxin_acyltransferase/base_genomes/temp_neg_genomes/GCF_002330425.1_ASM233042v1_genomic.fna
Sequence type: Nucleotide
Keep MBits: T
Maximum file size: 1000000000B
Adding sequences from FASTA; added 99 sequences in 

Adding sequences from FASTA; added 52 sequences in 0.154754 seconds.


Building a new DB, current time: 11/09/2021 11:30:26
New DB name:   /media/manu/RiPP_Prioritiser/RTX_toxin_acyltransferase/base_genomes/quality_control/neg_blast/databases/GCF_008502035.1_db
New DB title:  /media/manu/RiPP_Prioritiser/RTX_toxin_acyltransferase/base_genomes/temp_neg_genomes/GCF_008502035.1_CC2_genomic.fna
Sequence type: Nucleotide
Keep MBits: T
Maximum file size: 1000000000B
Adding sequences from FASTA; added 31 sequences in 0.133653 seconds.


Building a new DB, current time: 11/09/2021 11:30:26
New DB name:   /media/manu/RiPP_Prioritiser/RTX_toxin_acyltransferase/base_genomes/quality_control/neg_blast/databases/GCF_003449125.1_db
New DB title:  /media/manu/RiPP_Prioritiser/RTX_toxin_acyltransferase/base_genomes/temp_neg_genomes/GCF_003449125.1_ASM344912v1_genomic.fna
Sequence type: Nucleotide
Keep MBits: T
Maximum file size: 1000000000B
Adding sequences from FASTA; added 228 sequences in 0.101821 s

UnboundLocalError: local variable 'child' referenced before assignment

In [ ]:
# Use hmmer to find tp_gene contamination in negative samples
# This is to some extent dependent on which tp genes are chosen. If fewer neg_genomes result from the decontamination
#     process than are specified by select_neg_genomes, it might be worth trying a different set of tp_genes

with open(BGC_path+'/'+'report_2_generate_tp.txt', 'a') as f:
    f.write('\n'+'Using hmmer to check negative genomes for contamination'+'\n\n')

# Build net alignment of tp_genes
!muscle -in "{BGC_path}"/"{BGC_type}"_selected_tp_genes_altered.fasta -out "{neg_hmm_db_path}"/"{BGC_type}"_selected_tp_gene_alignment.fasta -loga "{BGC_path}"/report_2_generate_tp.txt

with open(BGC_path+'/'+'report_2_generate_tp.txt', 'a') as f:
    f.write('\n'+'Building nucleotide hmm of selected true positive genes'+'\n\n'+'Searching neg genomes with nucleotide hmm.\n\n')

# Build nucleotide based hmm of tp_genes alignment
!hmmbuild -n tp_nucl_aln --dna "{neg_hmm_db_path}"/"{BGC_type}"_tp_nucl.hmm "{neg_hmm_db_path}"/"{BGC_type}"_selected_tp_gene_alignment.fasta

# Search neg genomes with hmm
for genomes in neg_genomes_list:
    #metaBGC uses these cutoffs for all the protein hmms. This is more stringent/produces fewer results
    !nhmmer --F1 0.02 --F2 0.02 --F3 0.02 --tblout "{neg_hmm_results_path}"/"{genomes}"_hmm_result.tbl "{neg_hmm_db_path}"/"{BGC_type}"_tp_nucl.hmm "{neg_genomes_path}"/"{genomes}"*
    #This produces more results, should there be a lack of negative genomes.
#    !nhmmer --tblout "{neg_hmm_results_path}"/"{genomes}"_hmm_result.tbl "{neg_hmm_db_path}"/"{BGC_type}"_tp_nucl.hmm "{neg_genomes_path}"/"{genomes}"*

print('Done')

In [ ]:
# Parse hmm output
#https://stackoverflow.com/questions/62012615/convert-a-hmmer-tblout-output-to-a-pandas-dataframe

sample_list = []
id_list=[]
len_list=[]
evalue_list=[]


#Can't parse mhmmer output with biopython at the moment, so did parsing manually.
for filename in os.listdir(neg_hmm_results_path):
    samplename = '_'.join(filename.split('_')[0:2])
    with open(neg_hmm_results_path+'/'+filename) as handle:
        content = handle.readlines()
        for line in content:
            if not line.startswith('#'):
                sample_list.append(samplename)
                id_list.append(line.split()[0])
                len_list.append(int(float(line.split()[5])-float(line.split()[12])))
                evalue_list.append(float(line.split()[12]))

                
       
# Generate hit_df based on found tp_hits in neg genomes
hmm_dict = {'sample': sample_list, 'target_name': id_list, 'length': len_list,  'evalue': evalue_list}             
hit_df = pd.DataFrame.from_dict(hmm_dict)
hit_df.to_csv(neg_hmm_path+'/all_nhmmer_fn_samples.csv', index=False)

#filter hit_df by evalue cutoff
evalue_filter = hit_df['evalue'] <= hmm_evalue_cutoff
cutoff_hit_df = hit_df[evalue_filter]

#deduplicate samples
unique_hit_df = cutoff_hit_df.drop_duplicates(subset='sample', inplace=False)
unique_hit_df.to_csv(quality_control_path+'/nhmmer_elt'+str(hmm_evalue_cutoff)+'_fn_samples.csv', index=False)

# Generate a list of contaminated negative genomes with combined results from blast and hmm hits
hmm_remove_list = list(unique_hit_df.iloc[:,0])
remove_list = blast_remove_list
remove_list.extend(x for x in hmm_remove_list if x not in remove_list)

with open(BGC_path+'/'+'report_2_generate_tp.txt', 'a') as f:
    f.write('\n'+'Contamination with tp_seqs found in samples:\n'+str(hmm_remove_list)+'\n\n'+'Found '+str(len(remove_list))+' total samples with tp contamination in neg genomes.')

# Remove these contaminated samples from the possible pool of negative sequences
cleaned = ~unique_neg_df['assembly'].isin(remove_list)
cleaned_unique_neg_df = unique_neg_df[cleaned]

# Select a preset number of negative genomes (this could lead to fewer genomes available than selected. Handle)
selected_neg_genomes = list(cleaned_unique_neg_df.iloc[:,1])[0:select_neg_genomes]

# report block
with open(BGC_path+'/'+'report_2_generate_tp.txt', 'a') as f:
    f.write('\n\n decontaminated selected_neg_genomes are:\n')
    f.write(str(selected_neg_genomes))

for genome in selected_neg_genomes:
    print('moving negative', genome, 'to', output_neg_path)
    !mv "{neg_genomes_path}"/"{genome}"* "{output_neg_path}"
    
print('Done')

In [ ]:
# To confirm, run blast on all samples selected as pos genomes

# makes blast databases of all individual neg genomes (easier to keep track of accession numbers than when combining)
for genomes in selected_tp_genomes:
    !makeblastdb -in "{output_pos_path}"/"{genomes}"* -dbtype nucl -out "{pos_blast_db_path}"/"{genomes}"_db

for genomes in selected_tp_genomes:
    !blastn -db "{pos_blast_db_path}"/"{genomes}"_db -query "{BGC_path}"/"{BGC_type}"_selected_tp_genes_altered.fasta -out "{pos_blast_results_path}"/"{genomes}".blastout -outfmt "6 qseqid sseqid pident evalue length"
    
# use pandas to concatenate all blast output tables
df_list = []
for outfile in os.listdir(pos_blast_results_path):
    try:
        blast_df = pd.read_csv(pos_blast_results_path+'/'+outfile, sep='\t', names=['qseqid', 'sseqid', 'pident', 'evalue', 'length'], index_col=None)
        blast_df['sample'] = '.'.join(outfile.split('.')[0:2])
        df_list.append(blast_df)
    except EmptyDataError:
        continue
        
# Generate a list of contaminated negative genomes        
pos_blast_hits_df = pd.concat(df_list)
pos_blast_hits_df.to_csv(pos_blast_path+'/blast_results_summary.csv', index=False)

In [ ]:
# To confirm, run nhmmer on all samples selected as pos genomes


# Search neg genomes with hmm
for genomes in selected_tp_genomes:
    !nhmmer --F1 0.02 --F2 0.02 --F3 0.02 --tblout "{pos_hmm_results_path}"/"{genomes}"_hmm_result.tbl "{neg_hmm_db_path}"/"{BGC_type}"_tp_nucl.hmm "{output_pos_path}"/"{genomes}"*

sample_list = []
id_list=[]
evalue_list=[]
len_list = []

#Can't parse mhmmer output with biopython at the moment, so did parsing manually.
for filename in os.listdir(pos_hmm_results_path):
    samplename = '_'.join(filename.split('_')[0:2])
    with open(pos_hmm_results_path+'/'+filename) as handle:
        content = handle.readlines()
        for line in content:
            if not line.startswith('#'):
                sample_list.append(samplename)
                id_list.append(line.split()[0])
                len_list.append(int(float(line.split()[5])-float(line.split()[12])))
                evalue_list.append(float(line.split()[12]))

                
       
# Generate hit_df based on found tp_hits in neg genomes
hmm_dict = {'sample': sample_list, 'target_name': id_list, 'length': len_list, 'evalue': evalue_list}             
hit_df = pd.DataFrame.from_dict(hmm_dict)
hit_df.to_csv(pos_hmm_path+'/nhmmer_all_pos_samples.csv', index=False)

#deduplicate samples
unique_hit_df = hit_df.drop_duplicates(subset='sample', inplace=False)
unique_hit_df.to_csv(pos_hmm_path+'/nhmmer_unique_pos_samples.csv', index=False)

print('Done')